Detta är en snabbguide över använding av Pythons ***Pandas*** för bearbetning av data. 

**Begrepp**  
aggregation -   
Concatenate -  
Subset -   
Groupby -   
Series VS Dataframe - 
Transponera -   
Merge -  "queries operations" 
katetiska produkten?? 
bins - intervall
corr() - korrelation

**Läsa in och visualisera data**

Läsa in data:  

In [ ]:
import pandas as pd 

# läsa in csv-fil
df = pd.read_csv("/mnt/data/Pisa mean perfromance scores 2013 - 2015 Data.csv") 

Visualisera data:

In [ ]:
df.shape           # (rader, kolumner)
df.columns         # kolumnnamn
df.head(3)         # första raderna
df.dtypes          # datatyper per kolumn
df.sample(3)       # slumpmässig titt
df.describe()      # sammanfattande statistik
df['2015 [YR2015]']  # en kolumn
df[['Country', '2015 [YR2015]']]  # flera kolumner
df.iloc[0]         # första raden
df.iloc[0:3]       # första tre raderna
df[df['2015 [YR2015]'] > 500]  # filtrera rader
df.sort_values(by='2015 [YR2015]', ascending=False)  # sortera efter kolumn
df.groupby('Country')['2015 [YR2015]'].mean()  # gruppera och aggregera
df.merge(other_df, on='Country')  # slå ihop med annan DataFrame



Välja kolumner & filtrera rader (Plockar ut relevant subset.)

In [ ]:
df[["Country Name", "Series Name", "2015 [YR2015]"]]         # välj kolumner
df[df["Country Name"].isin(["Sweden", "Norway"])]            # filter på lista
df[df["Series Name"].str.contains("Math", case=False, na=False)]  # textfilter


Byta namn på kolumner (enkelt & konsekvent)

In [ ]:
df = df.rename(columns=lambda c: c.strip())  # trimmar blanksteg
df = df.rename(columns={"Country Name": "country", "Series Name": "series"})


Ta bort dubbletter & tomma (Städa bort bruset)

In [ ]:
df = df.drop_duplicates()
df = df.dropna(subset=["series"])         # släng rader där 'series' saknas
df["country"] = df["country"].fillna("Unknown")  # fyll tomma med standard


Rensa till numeriskt (tåligt för “..”, kommatecken m.m.). Konverterar “siffror som text” till riktiga tal. Icke-tolkningsbara blir NaN.

In [ ]:
import re
def to_num(s):
    s = (s.astype(str)
           .str.replace(r"[^\d\-,.]", "", regex=True)  # bara siffror, minus, , .
           .str.replace(",", ".", regex=False))        # svenskt komma → punkt
    return pd.to_numeric(s, errors="coerce")

df["val_2015"] = to_num(df["2015 [YR2015]"])


Gruppning & aggregation (t.ex. medel per land/serie). 

In [ ]:
(df
 .groupby(["country", "series"], as_index=False)["val_2015"]
 .agg(["mean", "min", "max"])
 .reset_index())


Spara rensat resultat som CSV. 

In [ ]:
df.to_csv("pisa_clean.csv", index=False)


Spara som CSV:

In [ ]:
df.to_csv('output.csv', index=False)  # spara till CSV 


**exempel på ett workflow**

1) Läs in

In [ ]:
import pandas as pd

# Datafilen heter 'perfromance' (med r och f omkastade)
path = "/mnt/data/Pisa mean perfromance scores 2013 - 2015 Data.csv"
df = pd.read_csv(path)


2) Förstå vad filen innehåller 
   
Tolka resultatet: Du bör se kolumner som Country Name, Series Name, 2013 [YR2013], 2014 [YR2014], 2015 [YR2015].

In [ ]:
df.shape               # hur många rader/kolumner?
df.columns.tolist()    # vilka kolumner (länder, serier, år)?
df.head(5)             # snabb titt
df.dtypes              # vilka datatyper?


3) Standardisera kolumnnamn (frivilligt men bra)

In [ ]:
df = df.rename(columns=lambda c: c.strip())
df = df.rename(columns={"Country Name": "country", "Series Name": "series"})


4) Identifiera årskolumner automatiskt

In [ ]:
year_cols = [c for c in df.columns if "[" in c and "]" in c and c[:4].isdigit()]
year_cols  # t.ex. ['2013 [YR2013]', '2014 [YR2014]', '2015 [YR2015]']


5) Gör årskolumner numeriska (utan att skriva massa kod)  
  
Varför? För att kunna räkna ut medel/sortera korrekt.

In [ ]:
def to_num(s):
    s = (s.astype(str)
           .str.replace(r"[^\d\-,.]", "", regex=True)
           .str.replace(",", ".", regex=False))
    return pd.to_numeric(s, errors="coerce")

for c in year_cols:
    df[c] = to_num(df[c])


6) Filtrera till en serie (t.ex. “Mathematics performance (PISA)”)

In [ ]:
sub = df[df["series"].str.contains("math", case=False, na=False)].copy()

7) Sortera på ett år (t.ex. 2015) och visa topp-10 länder

In [ ]:
top10_2015 = sub[["country", "2015 [YR2015]"]].sort_values("2015 [YR2015]", ascending=False).head(10)
top10_2015


9) Rensa baserat på behov

Ta bort rader utan land/serie:

In [ ]:
sub = sub.dropna(subset=["country", "series"])

Ta bort dubbletter:

In [ ]:
sub = sub.drop_duplicates()

Fyll saknade årsvärden med t.ex. median:

In [ ]:
for c in year_cols:
    sub[c] = sub[c].fillna(sub[c].median())

10) Välj bara det du vill spara/visa

In [ ]:
result = sub[["country", "series"] + year_cols]
result.head()

11) Spara rensad/kuraterad data

In [ ]:
result.to_csv("pisa_clean_math_2013_2015.csv", index=False)

12) Enkel visualisering (valfritt, minimal kod)

Topplista 2015 med pandas inbyggt:

In [ ]:
top = (sub[["country", "2015 [YR2015]"]]
       .dropna()
       .sort_values("2015 [YR2015]", ascending=False)
       .head(15))
ax = top.plot(kind="barh", x="country", y="2015 [YR2015]", title="PISA Math – 2015 (Top 15)")
ax.invert_yaxis()


Utveckling 2013–2015 för ett land:

In [ ]:
swe = sub[sub["country"].eq("Sweden")][year_cols].T
swe.columns = ["Sweden"]
swe.plot(title="Sweden – Math (2013–2015)")


**Snabb “copy-paste”-mall (5 rader → klar för analys):**

In [ ]:
import pandas as pd, re
to_num = lambda s: pd.to_numeric(s.astype(str).str.replace(r"[^\d\-,.]", "", regex=True).str.replace(",", ".", regex=False), errors="coerce")
df = pd.read_csv("/mnt/data/Pisa mean perfromance scores 2013 - 2015 Data.csv").rename(columns={"Country Name":"country","Series Name":"series"})
year_cols = [c for c in df.columns if c[:4].isdigit()]
for c in year_cols: df[c] = to_num(df[c])
(df[df["series"].str.contains("math", case=False, na=False)]
   .sort_values("2015 [YR2015]", ascending=False)
   [["country","2015 [YR2015]"]].head(10))


**Boxplot**  
  
sammanfattning  
***Symbol.........................Betydelse***  
Box.................................mitten 50 % av datan (Q1–Q3)  
Linje i boxen...............median (mittenvärde)  
Whiskers(sträck)........normala värden inom 1.5 × IQR  
Prickar utanför...........outliers (ovanligt höga/låga värden)  
  
Vad en boxplot visar:  
En boxplot (låddiagram) visar hur värdena är fördelade — alltså:  
var mitten ligger,  
hur spridda värdena är,  
om det finns outliers (avvikande värden).  
Den bygger på fem nyckeltal ur datan (så kallade femnumrerssammanfattningen):  
  
Min, Q1 (25%), Median (50%), Q3 (75%), Max

In [ ]:
# Exempel på boxplot (kan inte visas här)
college.boxplot(column="Outstate", by="Private", figsize=(4,4))